In [10]:
import os
import cv2
import random
import numpy as np
import openslide
import matplotlib.pyplot as plt
from skimage.color import rgb2hsv
from skimage.filters import threshold_otsu
from scipy import ndimage

from Aslide import aslide

In [11]:
def maxpooling(img, kernel_size=11):
    h, w = img.shape
    img_out = np.ones((h, w))
    window_size = kernel_size // 2
    for j in range(window_size, h-window_size):
        for i in range(window_size, w-window_size):
            img_out[j][i] = np.max(img[j-window_size:j+window_size, i-window_size:i+window_size])
    return img_out

In [21]:
class FindROI:
    def __init__(self, image, kernel_size=5):
        self.image = image
        self.kernel_size = kernel_size
        
        kernel = np.ones((kernel_size, kernel_size), np.uint8)
        img_open = cv2.morphologyEx(image, cv2.MORPH_ERODE, kernel, iterations=5)
        img_open = cv2.morphologyEx(img_open, cv2.MORPH_DILATE, kernel, iterations=3)

        tissue_mask = img_open > threshold_otsu(img_open)
        img_out = self.maxpooling(tissue_mask)
        img_out = self.maxpooling(img_out)

        img, contours, hierarchy = cv2.findContours(img_out.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        print(len(contours))
        
        self.contour = contours[1]
        
    def maxpooling(self, img):
        h, w = img.shape
        img_out = np.ones((h, w))
        window_size = self.kernel_size // 2
        for j in range(window_size, h-window_size):
            for i in range(window_size, w-window_size):
                img_out[j][i] = np.max(img[j-window_size:j+window_size, i-window_size:i+window_size])
        return img_out
    
    def in_roi(self, x, y, w, h):
        pass

        
def find_roi(wsi_path, save_path):
    try:
        slide = openslide.OpenSlide(wsi_path)
    except:
        slide = aslide.Aslide(wsi_path)
    level = slide.level_count - 1

    source = np.array(slide.read_region((0, 0), level, slide.level_dimensions[level]).convert('RGB'))
    img_gray = cv2.cvtColor(source, cv2.COLOR_RGB2GRAY)
    slide.close()


    kernel_size = (5, 5)
    kernel = np.ones(kernel_size, np.uint8)
    img_open = cv2.morphologyEx(img_gray, cv2.MORPH_ERODE, kernel, iterations=5)
    img_open = cv2.morphologyEx(img_open, cv2.MORPH_DILATE, kernel, iterations=3)

    tissue_mask = img_open > threshold_otsu(img_open)
    img_out = maxpooling(tissue_mask, kernel_size=5)
    img_out = maxpooling(img_out, kernel_size=5)

    img, contours, hierarchy = cv2.findContours(img_out.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    print(len(contours))

    hull = cv2.convexHull(contours[1], False)
    cv2.drawContours(source, [hull], 0, color=(random.randint(0,255),random.randint(0,255),random.randint(0,255)), thickness=5)
#     plt.imshow(source)
    save_name = os.path.join(save_path, os.path.basename(wsi_path) + ".jpg")
    cv2.imwrite(save_name, source)

In [ ]:
data_path = "/home/nvme/batch6.4-added/WSIS-added"
wsi_paths = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith('.tif') or f.endswith('.kfb')]
save_path = "/home/nvme/batch6.4-added/WSIS-added-contours"

# wsi_path = "/home/nvme/batch6.4-added/WSIS-added/SZH00904.TMAP"
for wsi_path in wsi_paths:
    find_roi(wsi_path, save_path)

361
19
129
1099
41
15
1228
1088
23
18
608
2166
37
97
160
677
476
269
230
355
236
395
31
214
377
601
40
129
70
1053
449
21
370
2439
179
362
215
510
1118
441
19
277
5
824
131
19
1180
477
62
325
651
153
662
381
389
272
529
67
160
795
516
51
110
749
867
371
406
90
257
372
35
699
376
324
11
350
1313
328
792
33
378
376
342
200
221
1042
436
549
250
415
1279
99
275
330
724
722
411
840
588
422
373
171
280
354
527
256
436
90
71
389
369
390
581
437
794
460
125
80
1168
395
77
33
60
659
15
184
294
301
320
272
630
21
241
693
538
315
532
75
364
7
411
49
103
98
365
772
184
1231
501
280
80
414
32
66
78
116
313
409
71
421
291
308
254
287
109
840
476
1012
27
547
246
478
384
260
778
741
268
378
291
87
1569
923
278
758
922
226
61
629
75
474
601
213
289
8
19
48
160
332
1043
174
111
165
107
913
1174
319
61
445
16
57
477
292
1716
64
334
413
414
185
170
410
775
240
366
763
633
1119
638
1263
371
45
373
936
236
1027
28
636
457
1041
528
246
152
467
78
84
686
463
222
270
1142
208
82
68
659
584
514
1315
1022
601
42

In [ ]:
img_gray =cv2.cvtColor(np.array(slide.read_region((0, 0),
                       level,
                       slide.level_dimensions[level]).convert('RGB')), cv2.COLOR_RGB2GRAY)
                      
source = np.array(slide.read_region((0, 0),
                       level,
                       slide.level_dimensions[level]).convert('RGB'))
plt.imshow(img_gray, cmap="gray")

In [ ]:
kernel_size = (5, 5)
kernel = np.ones(kernel_size, np.uint8)
img_open = cv2.morphologyEx(img_gray, cv2.MORPH_ERODE, kernel, iterations=5)
img_open = cv2.morphologyEx(img_open, cv2.MORPH_DILATE, kernel, iterations=3)
# img_open = cv2.morphologyEx(img_open, cv2.MORPH_ERODE, kernel, iterations=1)
plt.imshow(img_open, cmap="gray")

In [ ]:
background = img_open > threshold_otsu(img_open)
tissue_mask = background.copy()
plt.imshow(tissue_mask, cmap="gray")

In [ ]:
def maxpooling(img, kernel_size=11):
    h, w = img.shape
    img_out = np.ones((h, w))
    window_size = kernel_size // 2
    for j in range(window_size, h-window_size):
        for i in range(window_size, w-window_size):
            img_out[j][i] = np.max(img[j-window_size:j+window_size, i-window_size:i+window_size])
    return img_out

In [ ]:
# tissue_mask = tissue_mask * 255
# img_out = maxpooling(tissue_mask, kernel_size=11)
img_out = maxpooling(tissue_mask, kernel_size=5)
img_out = maxpooling(img_out, kernel_size=5)

In [ ]:
plt.imshow(img_out, cmap="gray")

In [ ]:
img, contours, hierarchy = cv2.findContours(img_out.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)
print(len(contours))

In [ ]:
hull = cv2.convexHull(contours[1], False)

cv2.drawContours(source, [hull], 0, (0,0,255), 5)
plt.imshow(source)

In [ ]:
for i in range(10):
    hull = cv2.convexHull(contours[i], False)
    cv2.drawContours(source, [hull], 0, color=(random.randint(0,255),random.randint(0,255),random.randint(0,255)), thickness=5)
plt.imshow(source)